In [ ]:
# coding:utf-8
'''未能精确匹配数据原因导出及模糊匹配
'''
import csv
import time
import codecs


class CheckSingle():
    '''传入两个对比列表（元素按固定顺序[id,brand,factory,series,power,trans,year,model]）'''
    def __init__(self,f6_list,che_list):
        self.f6_list = f6_list
        self.che_list = che_list
        
        self.f6_id = f6_list[0]
        self.f6_brand = f6_list[1]
        self.f6_factory = f6_list[2]
        self.f6_series = f6_list[3]
        self.f6_power = f6_list[4]
        self.f6_trans = f6_list[5]
        self.f6_year = f6_list[6]
        self.f6_model = f6_list[7]
        
        self.che_id = che_list[0]
        self.che_brand = che_list[1]
        self.che_factory = che_list[2]
        self.che_series = che_list[3]
        self.che_power = che_list[4]
        self.che_trans = che_list[5]
        self.che_year = che_list[6]
        self.che_model = che_list[7]
        
    # 单独处理变速箱不一致
    def check_trans(self):
        self.f6_condition = self.f6_brand + self.f6_factory + self.f6_series + self.f6_power + self.f6_year
        self.che_condition = self.che_brand + self.che_factory + self.che_series + self.che_power + self.che_year
        # 1.判断除变速箱之外的条件是否符合
        if self.f6_condition == self.che_condition:
            # 2.Model判断
            f6_model_lists = model_split(self.f6_model)
            for f6_model_list in f6_model_lists:
                # 计数
                count_num = 0
                for model in f6_model_list:
                    # 遍历每个单词，符合条件（即，在车型名称中）则计数+1
                    if model in self.che_model:
                        count_num += 1
                # 当计数为 f6_model_list 的长度时，表示所有描述性单词都在车型名称中，符合条件
                if count_num == len(f6_model_list):
                    # 3.变速箱判断
                    if self.f6_trans == self.che_trans:
                        return [self.f6_id,self.che_id,'']
                    else:
                        return [self.f6_id,self.che_id,'变速箱不一致']   
        else:
            return None
        
    # 单独处理年份不一致
    def check_year(self):
        self.f6_condition = self.f6_brand + self.f6_factory + self.f6_series + self.f6_power + self.f6_trans
        self.che_condition = self.che_brand + self.che_factory + self.che_series + self.che_power + self.che_trans
        # 1.判断除年份之外的条件是否符合
        if self.f6_condition == self.che_condition:
            # 2.Model判断
            f6_model_lists = model_split(self.f6_model)
            for f6_model_list in f6_model_lists:
                # 计数
                count_num = 0
                for model in f6_model_list:
                    # 遍历每个单词，符合条件（即，在车型名称中）则计数+1
                    if model in self.che_model:
                        count_num += 1
                # 当计数为 f6_model_list 的长度时，表示所有描述性单词都在车型名称中，符合条件
                if count_num == len(f6_model_list):
                    # 3.年份判断
                    if self.f6_year == self.che_year:
                        return [self.f6_id,self.che_id,'']
                    elif abs(int(self.f6_year) - int(self.che_year)) < 4:
                        return [self.f6_id,self.che_id,'年份不一致（相差3年及以下）']
                    else:
                        return [self.f6_id,self.che_id,'年份不一致（相差4年及以上）']
        else:
            return None
    
    # 单独处理Model不一致
    def check_model(self):
        self.f6_condition = self.f6_brand + self.f6_factory + self.f6_series + self.f6_power + self.f6_trans + self.f6_year
        self.che_condition = self.che_brand + self.che_factory + self.che_series + self.che_power + self.che_trans + self.che_year
        # 1.判断除变速箱之外的条件是否符合
        if self.f6_condition == self.che_condition:
            # 2.Model判断
            f6_model_lists = model_split(self.f6_model)
            for f6_model_list in f6_model_lists:
                # 计数
                count_num = 0
                for model in f6_model_list:
                    # 遍历每个单词，符合条件（即，在车型名称中）则计数+1
                    if model in self.che_model:
                        count_num += 1
                # 当计数为 model_condition 的长度时，表示所有描述性单词都在车型名称中，符合条件
                if count_num == len(f6_model_list):
                    # 3.model判断
                    return [self.f6_id,self.che_id,'']
                else:
                    return [self.f6_id,self.che_id,'Model不符合']
        else:
            return None
        
def read_csv_file(file_name):
    '''读取csv文件，结果为以每行数据为列表（元素为每列属性）的总列表'''
    try:
        with open(file_name,'r') as file:
            contents = file.readlines()
    except FileNotFoundError:
        print('文件%s不存在' % file_name)
        return None
    else:
        f6 = [] 
        for line in contents:
            f6.append(line.strip().split(','))
    return f6

def model_split(model):
    '''分解F6的model为单词组成的列表'''
    if model.find('[') != -1 and model.find(']') != -1:
        model_1 = model[:model.find('[')].strip()
        model_2 = model[model.find('[')+1:model.find(']')].strip()

        model_1_list = model_1.split()
        model_2_list = model_2.split()
        model_list = [model_1_list,model_2_list]
    else:
        model_list = [model.split()]
    return model_list

# 处理文件名称
file_name_f6 = 'F6.csv'
file_name_f6_che = 'F6_CHE.csv'
# 读取文件
f6 = read_csv_file(file_name_f6)
f6_che = read_csv_file(file_name_f6_che)
# F6结果
f6_result = []

'''
# 测试示例
f6 = [
    ['978d683e6a97dbdb6b2ffbab28fa7476','奥迪','一汽-大众奥迪','奥迪A3','1.4T','双离合变速箱(DCT)','2019','A3 Limousine'],
    ['555d683e6a97dbdb6b2ffbab28fa888','奥迪','一汽-大众奥迪','奥迪A4','1.4T','双离合变速箱(DCT)','2019','A4L Limousine'],
]
f6_che = [
            ['777','奥迪','一汽-大众奥迪','奥迪A3','1.4T','双离合变速箱(DCT)','2019','daf sa dfa A3 Limousine ads fasd fasd'],
            ['888','奥迪','一汽-大众奥迪','奥迪A4','1.4T','双离合变速箱(DCT)','2018','daf sa dfa A4L Limousine ads fasd fasd'],
         ]
'''

# 遍历F6
count_print = 0
for i in f6:
    count_print += 1
    print('第%s条记录（共%s条）正在匹配' % (count_print,len(f6)))   
    # 遍历车型库
    for j in f6_che:
        # 创建类实例
        checks = CheckSingle(i,j)
        # 分别处理变速箱，年款，model 不一致情况
        check_trans_result = checks.check_trans()
        check_year_result = checks.check_year()
        check_model_result = checks.check_model()
        single_id_result = [check_trans_result,check_year_result,check_model_result]
        for k in single_id_result:
            if k and (k not in f6_result):
                # 加入F6结果
                f6_result.append(k)

print('车型匹配处理完毕')
print(f6_result)

In [1]:
# 文件处理
import csv
import time
import codecs

def read_csv_file(file_name):
    '''读取csv文件，结果为以每行数据为列表（元素为每列属性）的总列表'''
    try:
        with open(file_name,'r') as file:
            contents = file.readlines()
    except FileNotFoundError:
        print('文件%s不存在' % file_name)
        return None
    else:
        results = [] 
        for line in contents:
            results.append(line.strip().split(','))
    return results

def utf8_2_gbk(oldfile):
    # 设置新文件名称
    newfile = "gbk_%s" % oldfile
    with codecs.open(oldfile, 'r', 'utf-8') as f:
        utf_str = f.read()
        out_gbk_str = utf_str.encode('GB18030')
    with open(newfile,'wb') as ff:
        ff.write(out_gbk_str)
        
def che_write(deal_with_dict,deal_with_file):
    with open('deal_%s'% deal_with_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        for k,v in deal_with_dict.items():
            writer.writerow([k,str(v)])
        
deal_with_file = 'not_year_3+.csv'
read_result = read_csv_file(deal_with_file)

f6_deal_with_dict = {}
che_deal_with_dict = {}

for i in read_result:
    # F6多值处理
    if i[0] in f6_deal_with_dict:
        f6_deal_with_dict[i[0]].append(i[1])
    else:
        f6_deal_with_dict[i[0]] = [i[1]]
        
    # 车型库多值处理
    if i[1] in che_deal_with_dict:
        che_deal_with_dict[i[1]].append(i[0])
    else:
        che_deal_with_dict[i[1]] = [i[0]]

che_write(f6_deal_with_dict,'f6_%s' % deal_with_file)
che_write(che_deal_with_dict,'che_%s' % deal_with_file)

time.sleep(3)

utf8_2_gbk('deal_f6_%s' % deal_with_file)
utf8_2_gbk('deal_che_%s' % deal_with_file)
print('Game Over!')

Game Over!
